## Logistic Regression

*   Logistic Regression 
$$f(\textbf{x})=\sigma(\textbf{w}^T\textbf{x}+b)=\dfrac{1}{1+e^{-(\textbf{w}^T\textbf{x}+b)}}$$

*   Loss function : log loss 
$$\text{Loss}(\textbf{w})=-\dfrac{1}{N}\sum y\text{log}(f(\textbf{x}))+(1-y)\text{log}(1-f(\textbf{x})) $$   

*    Gradient Descent
$$\textbf{w}=\textbf{w}-\alpha \dfrac{\partial}{\partial\textbf{w}}\text{Loss}(\textbf{w})$$

In [9]:
import pandas as pd
import numpy as np
# from joblib import dump, load.

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as opt

import warnings
warnings.filterwarnings(action='ignore')

In [10]:
x_data = [[1, 2], [2, 1], [3, 2], [4, 3], [5, 1], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [11]:
# Initialize the weight and bias.
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# Set the optimizer you like.
optimizer = opt.SGD([W, b], lr=1)  # with learning-rate

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    # Calculate the cost (loss function).
    hypothesis = torch.sigmoid(x_train.matmul(W) + b)
    cost = -(y_train * torch.sigmoid(hypothesis) + 
            (1 - y_train) * torch.log(1 - hypothesis)).mean()
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch%100 == 0:
        print("Epoch {:4d}/{} Cost: {:.6f}".format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost: 0.035344
Epoch  100/1000 Cost: -0.248272
Epoch  200/1000 Cost: -0.249123
Epoch  300/1000 Cost: -0.249412
Epoch  400/1000 Cost: -0.249557
Epoch  500/1000 Cost: -0.249645
Epoch  600/1000 Cost: -0.249703
Epoch  700/1000 Cost: -0.249745
Epoch  800/1000 Cost: -0.249777
Epoch  900/1000 Cost: -0.249801
Epoch 1000/1000 Cost: -0.249821


## Logistic Regression with Real Data

In [12]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as opt

# Import the diabetes data.
diabetes = pd.read_csv('./diabetes.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [13]:
len(diabetes)

768

In [14]:
size = int(len(diabetes)*0.8)
train = diabetes[:size]
test = diabetes[size:]
print(train.groupby('Outcome').size()) # Imbalanced dataset

Outcome
0    401
1    213
dtype: int64


In [15]:
# Normalize the input data.
X_train = np.asarray(train.drop('Outcome', 1))
y_train = np.asarray(train['Outcome'])
X_test = np.asarray(test.drop('Outcome', 1))
y_test = np.asarray(test['Outcome'])

mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)

X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

In [16]:
# If you want to balance the training dataset...
# You can skip this part!

# Split each datasets depending on the target data.
X_train_pos = X_train[y_train == 0]
X_train_neg = X_train[y_train == 1]
y_train_pos = y_train[y_train == 0]
y_train_neg = y_train[y_train == 1]

ids = np.arange(len(X_train_neg))
choices = np.random.choice(ids, len(X_train_pos))

X_train_neg = X_train_neg[choices]
y_train_neg = y_train_neg[choices]
X_train = np.concatenate([X_train_pos, X_train_neg], axis=0)
y_train = np.concatenate([y_train_pos, y_train_neg], axis=0)

# Shuffle the data
order = np.arange(len(X_train))
np.random.shuffle(order)
X_train = X_train[order]
y_train = y_train[order]

In [17]:
# Convert into tensor type.
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
# The target data should keep one dimension(Rank 1) -> unsqueeze it!
y_train = torch.FloatTensor(y_train).unsqueeze(-1)
y_test = torch.FloatTensor(y_test).unsqueeze(-1)

In [18]:
# Train the Model with F.binary_cross_entropy

# Initialize the Weight and the Bias of the Model.
W = torch.zeros((8, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# Set an Optimizer. 
#    > Stochastic Gradient Descent
#    > Learning Rate: 1
optimizer = opt.SGD([W, b], lr=1)
epochs = 100

for epoch in range(epochs+1):
    # Cost : Sigmoid
    hypothesis = torch.sigmoid(X_train.matmul(W) + b)
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    # Change the function.
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # Check the log.
    if epoch%20==0:
        print("Epoch {:4d}/{} ----- Cost: {:4f}".format(epoch, epochs, cost.item()))

Epoch    0/100 ----- Cost: 0.693147
Epoch   20/100 ----- Cost: 0.505788
Epoch   40/100 ----- Cost: 0.504883
Epoch   60/100 ----- Cost: 0.504847
Epoch   80/100 ----- Cost: 0.504845
Epoch  100/100 ----- Cost: 0.504845


In [19]:
# Test
hypothesis = torch.sigmoid(X_test.matmul(W) + b)

threshold = 0.5
prediction = hypothesis >= torch.FloatTensor([threshold])

correct = prediction.float() == y_test
accuracy = correct.sum().item() / len(correct)

In [20]:
print("Test accuracy: {:2.2f}%".format(accuracy*100))

Test accuracy: 75.32%
